## A liga ternária mais cara do mundo


## Introdução



<p style="text-align: justify"> textinho legal sobre ligas.txt</p>

## Objetivo



<p style="text-align: justify">Encontre uma liga de três elementos que tenha o maior custo possível. A liga ternária deve ser da forma $x$A.$y$B.$z$C sendo que $x+y+z = 100\,\mathrm{g}$, $x>5\,\mathrm{g}$, $y>5\,\mathrm{g}$, $z>5\,\mathrm{g}$ e &ldquo;A&rdquo;, &ldquo;B&rdquo; e &ldquo;C&rdquo; são elementos químicos. Considere que qualquer composto com 3 elementos químicos é chamado de liga.</p>


<h3> Desenvolvimento do algortimo </h3>

<p style="text-align: justify"> Para resolver este problema, é necessário pensar primeiramente no formato dos individuos que serão gerados e mutados. Seguindo as dicas dadas, o método desenvolvido trabalhará com listas referentes as caracteristicas dos genes/elementos, afim de simplificar a iteração. </p>
 
<h4> Parametros iniciais </h4>
<ul>
    <li>A liga deve pesar 100g </li>
    <li>Os pesos e valores são determinados por uma lista pré-existente (espaço de busca)</li>
    <li>A parcela de cada elemento na liga deve ser maior que 5%</li>
</ul>

<h4> Passos</h4>
<ul>
    <li> Selecionar genes, que serão elementos da lista</li>
    <li> Gerar individuo válido </li>
    <li> Gerar uma população </li>
    <li> Avaliar seu respectivo preço (fitness_ind e fitness_pop) </li>
    

## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



<p style="text-align: justify"> Para resolver este problema, é preciso aplicar restrições ao algoritmo, de forma que ele obedeça aos parâmetros postos. Desta forma, cria-se primeiro indivíduos válidos. O problema fornece um dicionário onde as chaves são os nomes dos elementos e os valores correspondem preço por kilograma (neste caso, talvez seja conveniente converter a unidade para gramas, a unidade que será trabalhada). Desta forma, falta determinar quanto de cada elemento estará presente na liga, uma forma de fazer isto é com uma lista de valores que será associada de forma aleatória aos elementos selecionados, após os elementos que compõe a liga (indivíduo) carregarem todas as propriedades, aplica-se a restrição de massa_total = 100g, se o indivíduo não atende a este critério, ele não é valido. </p>


In [3]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 100

# preço em dólares por kilograma
ELEMENTOS = {
    "H": 1.39,
    "He": 24,
    "Li": 85.6,
    "Be": 857,
    "B": 3.68,
    "C": 0.122,
    "N": 0.14,
    "O": 0.154,
    "F": 2.16,
    "Ne": 240,
    "Na": 3.43,
    "Mg": 2.32,
    "Al": 1.79,
    "Si": 1.7,
    "P": 2.69,
    "S": 0.0926,
    "Cl": 0.082,
    "Ar": 0.931,
    "K": 13.6,
    "Ca": 2.35,
    "Sc": 3460,
    "Ti": 11.7,
    "V": 385,
    "Cr": 9.4,
    "Mn": 1.82,
    "Fe": 0.424,
    "Co": 32.8,
    "Ni": 13.9,
    "Cu": 6,
    "Zn": 2.55,
    "Ga": 148,
    "Ge": 1010,
    "As": 1.31,
    "Se": 21.4,
    "Br": 4.39,
    "Kr": 290,
    "Rb": 15500,
    "Sr": 6.68,
    "Y": 31,
    "Nb": 85.6,
    "Mo": 40.1,
    "Tc": 100000,
    "Ru": 10600,
    "Rh": 147000,
    "Pd": 49500,
    "Ag": 521,
    "Cd": 2.73,
    "In": 167,
    "Sn": 18.7,
    "Sb": 5.79,
    "Te": 63.5,
    "I": 35,
    "Xe": 1800,
    "Cs": 61800,
    "Ba": 0.275,
    "La": 4.92,
    "Ce": 4.71,
    "Pr": 103,
    "Nd": 57.5,
    "Pm": 460000,
    "Sm": 13.9,
    "Eu": 31.4,
    "Gd": 28.6,
    "Tb": 658,
    "Dy": 307,
    "Ho": 57.1,
    "Er": 26.4,
    "Tm": 3000,
    "Yb": 17.1,
    "Lu": 643,
    "Hf": 900,
    "Ta": 312,
    "W": 35.3,
    "Re": 4150,
    "Os": 12000,
    "Ir": 56200,
    "Pt": 27800,
    "Hg": 30.2,
    "Tl": 4200,
    "Pb": 2,
    "Bi": 6.36,
    "Po": 49200000000000,
    "Ac": 29000000000000,
    "Th": 287,
    "Pa": 280000,
    "U": 101,
    "Np": 660000,
    "Pu": 6490000,
    "Am": 750000,
    "Cm": 160000000000,
    "Bk": 185000000000,
    "Cf": 185000000000,
}

# convertendo valores para gramas
PRECO_POR_GRAMA = {}
for i in ELEMENTOS:
    a = ELEMENTOS[i]*0.001
    PRECO_POR_GRAMA[i] = a
    
NUM_OBJETOS = len(ELEMENTOS)
ORDEM_DOS_NOMES = list(sorted(ELEMENTOS.keys()))

In [6]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, ELEMENTOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [ ]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

## Conclusão



Delete este texto e escreva sua conclusão.



## Referências consultadas



1.  Delete este texto e inclua suas referências ordenadas numericamente. Se for referenciar no notebook, use o número entre colchetes (exemplo: para citar essa referência aqui escreva &ldquo;[1]&rdquo; sem as áspas).

2.  Cada item deve ser numerado. Siga o padrão apresentado.

3.  Caso não tenha nenhuma referência consultada, delete esta seção e o texto contido nela!



## Playground



In [ ]:
# ALTERAR FUNCAO 
# A FUNCÃO PARA NO if
# POR QUE?

def computa_mochila(individuo, objetos, ordem_dos_nomes):
    """Computa o valor total e peso total de uma mochila
    
    Args:
    
      individiuo: Lista binária contendo a informação de quais objetos serão selecionados.
      objetos: Dicionário onde as chaves são os nomes dos objetos e os valores são
               dicionários com a informação do peso e valor.
      ordem_dos_nomes: Lista contendo a ordem dos nomes dos objetos.
      
      
    Returns:
      valor_total: valor total dos itens da mochila em unidades de dinheiros.
      peso_total: peso total dos itens da mochila em unidades de massa.
      
    """
    massa = [random.randint(5, 100) for _ in range(92)] # lista relacionando um valor de massa para cada elemento
    valor_total = 0
    peso_total = 0 
    
    for pegou_o_item_ou_nao, nome_do_item, massa_do_elemento in zip(individuo, ordem_dos_nomes, massa):
        if pegou_o_item_ou_nao == 1:
            print(a)
            valor_do_item = objetos[nome_do_item]
            peso_do_item = massa_do_elemento
            #print(valor_do_item, peso_do_item)
            
            valor_total = valor_total + valor_do_item
            peso_total = peso_total+ peso_do_item

    return valor_total, peso_total